In [1]:
from src.sat_generator import URGenerator, SRGenerator
from src.solvers import random_solver, minisat_solver
from src import utils

import numpy as np
import copy
import random
import pandas as pd

2022-12-21 15:28:29.218477: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
# Create a sat generator
sat_gen = SRGenerator(n = 20, 
                      p_bernoulli = 0.7,
                      p_geometric = 0.4)

# Create a random sat formula
n, m, r, [formula_unsat, formula] = sat_gen.generate_formula()

print(f'n: {n}')
print(f'm: {m}')
print(f'r: {r}')
print()

print(formula)

In [2]:
# Create a sat generator
sat_gen = URGenerator(min_n = 5,
                      max_n = 5,
                      min_k = 3,
                      max_k = 3,
                      min_m = 20,
                      max_m = 20)

# Create a random sat formula
n, m, r, formula = sat_gen.generate_formula()

print(f'n: {n}')
print(f'm: {m}')
print(f'r: {r}')
print(formula)      

n: 5
m: 20
r: 4.0
[[4, 3, -5], [1, 3, -2], [1, 2, 5], [4, -3, -2], [-1, 3, 2], [-5, -3, 4], [-4, -5, -1], [-1, -5, -3], [1, -5, 4], [1, 2, 3], [2, -4, -3], [4, -1, -2], [-1, 4, 5], [-5, 2, 1], [5, 4, 3], [-1, 3, -4], [3, -5, 4], [-4, -2, 1], [1, -3, 5], [-3, 1, -4]]


In [ ]:
dimacs_path =  'data/uf20-91/uf20-01.cnf'
n, m, formula = utils.dimacs2list(dimacs_path = dimacs_path)

print(f'n: {n}')
print(f'm: {m}')
print(f'r: {m/n}')
print()
print(formula)

In [3]:
# Create a random assignment
assignment, num_sat = random_solver(n, formula)
print(f'assignment: {assignment}')
print(f'num of sat clauses: {num_sat} out of {m}')

assignment: [1 1 0 0 0]
num of sat clauses: 17 out of 20


In [4]:
assignment, is_sat = minisat_solver(n, formula)
print(f'assignment: {assignment}') # None means there is no assigment that satisfies the formula.
print(f'is_sat: {is_sat}')

assignment: [1, 1, 1, 1, 0]
is_sat: True


## WalkSAT

In [18]:
def flip(variable, assignment, formula):
    new_assignment = copy.deepcopy(assignment)
    # Flip the chosen variable
    if assignment[variable]:
        new_assignment[variable] = 0
    else:
        new_assignment[variable] = 1
    is_sat, new_num_sat, _ = utils.num_sat_clauses(formula, assignment)
    return is_sat, new_num_sat, new_assignment

    
def walksat(formula, n, max_flips, max_tries, p=0.57):
    for i in range(max_tries):
        assignment = utils.random_assignment(n=n)
        print("assignment selected random:", assignment)
        for j in range(max_flips):
            is_sat, num_sat, eval_formula = utils.num_sat_clauses(formula, assignment)
            
            #eval_formula = [int(elem) for elem in eval_formula]
            print(i,j, num_sat)
            if is_sat:
                return assignment
            # Count the break-count for each variable.
            for variable in range(n): #make the list ordered at random
                assignment_list = []
                num_sat_list = []

                is_sat, new_num_sat, new_assignment = flip(variable, assignment, formula)
                if is_sat:
                    return assignment
                assignment_list.append(new_assignment)
                num_sat_list.append(new_num_sat)
                if new_num_sat >= num_sat:
                    break_count_is_zero = True
                    break
                else:
                    break_count_is_zero = False

            # if there is a variable x with break-count x, choose x
            #if break_count_is_zero:
                # We keep the new_assigment (that is: flip the 'variable' in assignment)
                #assignment = new_assignment

            if not break_count_is_zero:
                random_num = random.uniform(0, 1)
                if random_num < p:
                    # Random walk. Flip a random variable in the clause
                    variable = random.randrange(n)
                    _, _, new_assignment = flip(variable, assignment, formula)
                else:
                    variable = pd.Series(num_sat_list).idxmin()
                    new_assignment = assignment_list[variable]

            assignment = new_assignment

    return str("Fail")

import time
start = time.time()
assignment = walksat(formula, 5, 10000, 4)
end = time.time()
print(end-start)
print(assignment)

assignment selected random: [0 0 0 1 1]
0 0 18
0 1 17
0 2 18
0 3 17
0 4 18
0 5 17
0 6 18
0 7 17
0 8 18
0 9 17
0 10 18
0 11 17
0 12 18
0 13 17
0 14 18
0 15 17
0 16 18
0 17 17
0 18 18
0 19 17
0 20 18
0 21 17
0 22 18
0 23 17
0 24 18
0 25 17
0 26 18
0 27 17
0 28 18
0 29 17
0 30 18
0 31 17
0 32 18
0 33 17
0 34 18
0 35 17
0 36 18
0 37 17
0 38 18
0 39 17
0 40 18
0 41 17
0 42 18
0 43 17
0 44 18
0 45 17
0 46 18
0 47 17
0 48 18
0 49 17
0 50 18
0 51 17
0 52 18
0 53 17
0 54 18
0 55 17
0 56 18
0 57 17
0 58 18
0 59 17
0 60 18
0 61 17
0 62 18
0 63 17
0 64 18
0 65 17
0 66 18
0 67 17
0 68 18
0 69 17
0 70 18
0 71 17
0 72 18
0 73 17
0 74 18
0 75 17
0 76 18
0 77 17
0 78 18
0 79 17
0 80 18
0 81 17
0 82 18
0 83 17
0 84 18
0 85 17
0 86 18
0 87 17
0 88 18
0 89 17
0 90 18
0 91 17
0 92 18
0 93 17
0 94 18
0 95 17
0 96 18
0 97 17
0 98 18
0 99 17
0 100 18
0 101 17
0 102 18
0 103 17
0 104 18
0 105 17
0 106 18
0 107 17
0 108 18
0 109 17
0 110 18
0 111 17
0 112 18
0 113 17
0 114 18
0 115 17
0 116 18
0 117 17
0 118 18

In [20]:
if type(assignment) != str:
    is_sat, num_sat, eval_formula = utils.num_sat_clauses(formula, assignment)
    print(f'assignment sat the formula: {is_sat}')

assignment sat the formula: True


## GSAT

In [21]:
def flip(assignment, formula, n):
    assignment_list = []
    num_sat_list = []

    for variable in range(n):
        new_assignment = copy.deepcopy(assignment)
        if assignment[variable]:
            new_assignment[variable] = 0
        else:
            new_assignment[variable] = 1
        #print(assignment)
        #print(new_assignment)
        is_sat, num_sat, eval_formula = utils.num_sat_clauses(formula, assignment)
        assignment_list.append(new_assignment)
        num_sat_list.append(num_sat)
    #print(assignment_list)
    #print(num_sat_list)
    max_value = max(num_sat_list)
    max_indexes = [i for i in range(n) if num_sat_list[i] == max_value]
    #print("max_indexes", max_indexes)
    max_index = random.choice(max_indexes)
    #print("max index", max_index)

    #print("max_asigment", assignment_list[max_index])
    return assignment_list[max_index]


def GSAT(formula, n, max_flips, max_tries):
    for i in range(max_tries):
        assignment = utils.random_assignment(n=n)
        print("assignment selected random:", assignment)
        for j in range(max_flips):
            is_sat, num_sat, eval_formula = utils.num_sat_clauses(formula, assignment)
            #eval_formula = [int(elem) for elem in eval_formula]
            print(i,j, num_sat)
            if is_sat:
                return assignment
            else:
                assignment = flip(assignment, formula, n)
                print(assignment)
    return str("Fail")


import time
start = time.time()
assignment = GSAT(formula, n, 10000, 4)
end = time.time()
print(end-start)
print(assignment)

assignment selected random: [1 1 1 0 0]
0 0 17
[1 1 0 0 0]
0 1 17
[1 1 0 1 0]
0 2 19
[0 1 0 1 0]
0 3 18
[0 1 1 1 0]
0 4 17
[0 1 1 1 1]
0 5 18
[0 1 1 1 0]
0 6 17
[1 1 1 1 0]
0 7 20
0.004394054412841797
[1 1 1 1 0]


In [22]:
if type(assignment) != str:
    is_sat, num_sat, eval_formula = utils.num_sat_clauses(formula, assignment)
    print(f'assignment sat the formula: {is_sat}')

assignment sat the formula: True
